# Random forests

**Notice: This notebook is a modification of [go.ipynb](https://mlbook.explained.ai/notebooks/index.html) by Terence Parr and Jeremy Howard. It is being used by permission of the author.**


## Looking at the data

In [ ]:
import pandas as pd

In [ ]:
rent = pd.read_csv('rent-ideal.csv')
rent.head()

In [ ]:
rent.shape

In [ ]:
rent.info()

## Training a random forest model

In [ ]:
X = rent.drop('price', axis=1)
y = rent['price']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
type(X)

In [ ]:
type(y)

In [ ]:
X.values

In [ ]:
y.values

In [ ]:
type(X.values)

In [ ]:
type(y.values)

The above *dataframe* and *array* versions of our data are both Python implementations (via Pandas and NumPy, respectively) of Figure 2.4 in the text:

![figure2.4](data_as_arrays.PNG)

where each row $\mathbf{X}_i$ represents a feature vector (a single apartment for our data here) and each $y_i$ represents the corresponding target (rent price for us). 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=10) 

![instantiate](Week_3_Code_Notes_1.jpeg)

In [ ]:
rf.fit(X.values, y.values)

![train](Week_3_Code_Notes_2.jpeg)

In [ ]:
import numpy as np

unknown_x = np.array([2, 1, 40.7957, -73.97])
unknown_x

![predict](Week_3_Code_Notes_3.jpeg)

In [ ]:
rf.predict([unknown_x])

In [ ]:
unknown_xs = np.array([[2, 1, 40.7957, -73.97],
                        [3, 2, 40.8123, -73.9654]])
unknown_xs

In [ ]:
rf.predict(unknown_xs)

In [ ]:
# prediction of each tree

tree_preds = [tree.predict(unknown_xs) for tree in rf.estimators_]
tree_preds

In [ ]:
test_pt_1 = 0
test_pt_2 = 0

for p in tree_preds:
    test_pt_1 += p[0]
    test_pt_2 += p[1]

pred1 = test_pt_1/len(tree_preds)
pred2 = test_pt_2/len(tree_preds)

In [ ]:
pred1, pred2

## Does the model capture training data relationships?

In [ ]:
X.values

In [ ]:
rf.predict(X.values)

In [ ]:
y.values

We will use *mean absolute error* to determine how well our model does at predicting rent prices for the both the data in the training set and, in the next section, for the validation set. 

Recall that the formula for mean absolute error is

$$
\rm{mean\;absolute\;error} = \frac{1}{n}\sum_{i=1}^{n} |y_i - \hat{y}_i |
$$
where $y_i$ represent the target values (that is, the real answers) and $\hat{y}_i$ represent the corresponding predictions. 



This is what the function `mean_absolute_error` will calculate for us. 

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
predictions = rf.predict(X.values)

e = mean_absolute_error(y, predictions)

ep = e*100 / y.mean()

print(f"${e:.0f} average error; {ep:.2f}% error")

## Checking model generality

In [ ]:
from sklearn.model_selection import train_test_split

X = rent[['bedrooms','bathrooms','latitude','longitude']]
y = rent['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
rf = RandomForestRegressor(n_estimators=10) 

rf.fit(X_train, y_train)

In [ ]:
validation_predictions = rf.predict(X_test)

validation_e = mean_absolute_error(y_test, validation_predictions)

validation_ep = validation_e * 100 / y.mean() 

print(f"${validation_e:.0f} average error; {validation_ep:.2f}% error")

## Feature importances

We can also visualize which features are most important for the model based on two methods: 
- column permutation 
- drop column

To calculate feature importances we will use the `rfpimp` package, since the sklearn function to do the same has [some issues](https://explained.ai/rf-importance/index.html). 

> This package does not work with Python 3.12, so I have provided a modified version called `rfpimp_MC.py`. 

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

X_df = pd.DataFrame(X, columns=cancer.feature_names)
y_df = pd.DataFrame(y, columns=['target'])

X_df_train, X_df_test, y_df_train, y_df_test = train_test_split(X_df, y_df, test_size=0.2, random_state=123)

rf_cancer = RandomForestClassifier()
rf_cancer.fit(X_df_train, y_df_train.values.ravel())

print(f"Training accuracy is {rf_cancer.score(X_df_train, y_df_train)}")
print(f"Test accuracy is {rf_cancer.score(X_df_test, y_df_test):.2f}")


In [ ]:
from rfpimp_MC import *  # normally you don't do this but this is a very small package

imp = importances(rf_cancer, X_df_test, y_df_test) # default is column permutation 
viz = plot_importances(imp)
viz.view()